# Modify Pothole Examples

This script creates the modified pothole examples based off the original pothole datasets' examples.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import random as rand
import cv2

In [ ]:
default_titles = ['Input Image', 'Input Depth', 'Measured Depth Values', 'True Mask', 'Predicted Mask']

def displayImagesBGR2RGB(display_list, title=default_titles):
    plt.figure(figsize=(15, 15))

    for i in range(len(display_list)):
        ##print(type(display_list[i]))
        ##print(f"display - INFO: display_list[{i}].shape = {display_list[i].shape}")
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        img = display_list[i]
        plt.imshow(img[:, :, ::-1])
        plt.axis('off')
    plt.show()

In [ ]:
# modifyExample needs to:
# Change all of the pixels to 0 or 1
# Determine left most 1, determine right most 1, w = r-l
# Determine top most 1, determine bottom most 1, h = b-t
# Determine the center, c = (l+w/2, t+h/2)
# Modify image such that its: (l-w/2, t-h/2) to (r+w/2, b+h/2) 
#   giving double w,h for the final image
def modifyExample(folder_path, item_name):
    print(f"Running modifyExample({folder_path}, {item_name})")
    left_path = join(join(folder_path, "images"), f"{item_name}-left.png")
    depth_path = join(join(folder_path, "depths"), f"{item_name}-depth.png")
    ground_path = join(join(folder_path, "ground"), f"{item_name}-ground.png")
    
    left = cv2.imread(left_path.replace("_modified", ""))
    depth = cv2.imread(depth_path.replace("_modified", ""))
    ground = cv2.imread(ground_path.replace("_modified", ""))
    print(ground.shape)
    #ground_single = ground[:, :, 0]
    #print(np.where(list(np.argmin(ground[:, :, 0], axis=0))!= 0)) # Finds the 0s
    #print(np.where(list(np.argmin(ground[:, :, 0], axis=1))!= 0)) # Finds the 0s
    width_wise = np.argmax(ground[:, :, 1], axis=0)
    height_wise = np.argmax(ground[:, :, 1], axis=1)
    #print(width_wise.shape)
    #print(height_wise.shape)
    width_wise = np.nonzero(width_wise)
    height_wise = np.nonzero(height_wise)
    lw = np.min(width_wise)
    th = np.min(height_wise)
    rw = np.max(width_wise)
    bh = np.max(height_wise)
    
    new_lw = lw-(rw-lw)
    new_rw = rw+(rw-lw)
    new_th = th-(bh-th)
    new_bh = bh+(bh-th)
    
    new_lw = max(0, new_lw)
    new_rw = min(ground.shape[1], new_rw)
    new_th = max(0, new_th)
    new_bh = min(ground.shape[0], new_bh)
    
    print(lw, rw, th, bh)
    print(new_lw, new_rw, new_th, new_bh)
    #print(lw, new_lw)
    #print(rw, new_rw)
    #print(th, new_th)
    #print(bh, new_bh)
    
    modified_ground = ground[new_th:new_bh, new_lw:new_rw, :]
    modified_depth = depth[new_th:new_bh, new_lw:new_rw, :]
    modified_left = left[new_th:new_bh, new_lw:new_rw, :]
    
    cv2.imwrite(ground_path, modified_ground)
    cv2.imwrite(depth_path, modified_depth)
    cv2.imwrite(left_path, modified_left)
    
    # https://stackoverflow.com/questions/50963283/python-opencv-imshow-doesnt-need-convert-from-bgr-to-rgb
    displayImagesBGR2RGB([depth, left, ground], ["depth", "left", "ground"])
    displayImagesBGR2RGB([modified_depth, modified_left, modified_ground], \
                  ["modified_depth", "modified_left", "modified_ground"])
    
    

In [ ]:
from os import listdir
from os.path import isfile, join

test_every = 5

print("Which folder should the modified images be created for?")
#print("data_sealed_road or data_unsealed_road")
#print("data_sealed_pothole or data_unsealed_pothole")
print("data_sealed_pothole_modified or data_unsealed_pothole_modified")
source_folder_path = input("Folder: ")
source_folder_path = f"../{source_folder_path}"

folder_path = join(source_folder_path.replace("_modified", ""), "images")
train_file_names = []
test_file_names = []

item_names = listdir(folder_path)
item_names.sort()
for i, item_name in enumerate(item_names):
    if isfile(join(folder_path, item_name)):
        example_name = item_name.replace('-left.png', '')
        modifyExample(source_folder_path, example_name)
print(f"Finished modifying {source_folder_path}")


